In [12]:
import datetime as dt
import time
import logging as log
from multiprocessing import Pool, TimeoutError

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb
%run Evaluation.ipynb


class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
    def start_regression_on_one_stock(self,symbol,regression_end_date):
        start = time.time()
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        section_df = section_df[section_df['d_s_datetime']<to_datetime(regression_end_date)]
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(stock_start_date,stock_end_date,self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            if(self.debug==True):
                print('\nsection '+start_date+'===>'+end_date)
            while(start_datetime<=end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                
                if(metric.is_today_open(cur_date)):
                    account.daily_audit(cur_date)

                    if(account.can_open_new_stock()==True):
                        if(operation.is_buy_point(cur_date)==True):
                            operation.open_opsition(cur_date)

                    if(account.has_shares()==True):
                        if(operation.is_sell_point(cur_date)==True):
                            operation.sell_stock(cur_date)

                start_datetime += dt.timedelta(days = 1)
                    
                        
        operation_df = operation.get_operations()
        
        evaluation = Evaluation(account,self.context,self.policy_name)
        evaluation_df = evaluation.get_evaluation_report(symbol,stock_start_date,stock_end_date,
                                                        self.policy_name,operation_df)

        end = time.time()
        if(self.debug==True):
            print(symbol+' regression cost time ' + str(round(end-start,1))+' seconds')
        return evaluation_df
    
    
    def start_regression(self,scale,regression_end_date,process_num):
        print('context:' + str(self.context))
        
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale)
        total = len(symbols) 
        
        pool = Pool(processes=process_num)
        tasks = []
        for symbol in symbols:
            task = pool.apply_async(self.start_regression_on_one_stock, (symbol,regression_end_date))
            tasks.append(task)
            
        for task in tasks:
            try:
                one_evaluation = task.get()
                evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
            except Exception as e:
                print(str(e))
        
        pool.close()
        pool.join()
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        return evaluation_df
    

In [13]:
regression = RocketRegression('rocket_break_baseline.json')

regression_end='2019-05-30'
scale=100
process_num=24
evaluation_df = regression.start_regression(scale,regression_end,process_num)
evaluation_df

context:{'short': 10, 'mid': 240, 'scale_start': 0, 'scale_end': 3, 'level': 'Day'}
build SZ#002504 break sections
start to analyse Day level SZ#002504 all break sections...
build SZ#002181 break sections
build SZ#002505 break sections
build SZ#000806 break sections
start to analyse Day level SZ#002181 all break sections...
build SZ#002020 break sections
build SZ#000803 break sections
build SZ#002021 break sections
start to analyse Day level SZ#000803 all break sections...
build SZ#000576 break sections
build SZ#002506 break sections
start to analyse Day level SZ#002020 all break sections...
build SZ#002182 break sections
start to analyse Day level SZ#000576 all break sections...
build SZ#000807 break sections
start to analyse Day level SZ#000807 all break sections...
start to analyse Day level SZ#002505 all break sections...
start to analyse Day level SZ#000806 all break sections...
build SZ#000581 break sections
start to analyse Day level SZ#002182 all break sections...
start to anal

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:165: RuntimeWarning: invalid value encountered in double_scalars


build SZ#002351 break sections
start to analyse Day level SZ#002351 all break sections...
build SZ#002512 break sections
start to analyse Day level SZ#002512 all break sections...
build SZ#000010 break sections
start to analyse Day level SZ#000010 all break sections...
build SZ#000590 break sections
start to analyse Day level SZ#000590 all break sections...


ZeroDivisionError: float division by zero

build SZ#000816 break sections
start to analyse Day level SZ#000816 all break sections...
build SZ#002029 break sections
start to analyse Day level SZ#002029 all break sections...
build SZ#002190 break sections
start to analyse Day level SZ#002190 all break sections...
build SZ#002352 break sections
start to analyse Day level SZ#002352 all break sections...
build SZ#002513 break sections
start to analyse Day level SZ#002513 all break sections...
build SZ#000011 break sections
start to analyse Day level SZ#000011 all break sections...
build SZ#000591 break sections
start to analyse Day level SZ#000591 all break sections...
build SZ#000818 break sections
start to analyse Day level SZ#000818 all break sections...
build SZ#002030 break sections
start to analyse Day level SZ#002030 all break sections...
build SZ#002191 break sections
start to analyse Day level SZ#002191 all break sections...
build SZ#002353 break sections
start to analyse Day level SZ#002353 all break sections...
build SZ#0

In [ ]:
evaluation_df.describe()